In [1]:
# 調整參數
fs = 400000 #400k

In [2]:
#load cyclos data
import csv
import numpy as np
real_datas = []
with open('cyclo2_fresh.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        real_datas.append( list( map(float,row) ) )
    csvfile.close()
real_datas = np.array(real_datas)

In [ ]:
# signal
import matplotlib.pyplot as plt
from scipy import stats
print( stats.describe(real_datas[0]) )
plt.figure(figsize=(20,10))
plt.plot(np.arange(0,len(real_datas[1])/fs,1/fs),real_datas[10])
plt.xlabel("Time")
plt.ylabel("Amp")

In [ ]:
# check frequency amplitude in (-1,1) downside

In [ ]:
from scipy import signal
#fs = 360000
plt.figure(figsize=(30,10))

cm=plt.cm.get_cmap('rainbow')
freq, times, Zxx = signal.stft(real_datas[0], fs=fs, nperseg=257, noverlap=128)
print(Zxx.shape)
# freq = freq[1:]
# Zxx = Zxx[1:]

# # handle abs 
# lownum = 1e-8
# Zxx2 = abs(Zxx) + lownum
# Zxx2log = np.log10(Zxx2)
# minva = -4
# Zxx2nor = ( Zxx2log - (minva) )/abs(minva)
# plt.pcolormesh(times, freq, Zxx2nor,cmap=cm)

# # handle the angle
# phase = np.angle(Zxx)
# finetuneangle = 3.15
# tophase = phase / finetuneangle


# print(np.max(Zxx2nor),np.min(Zxx2nor),np.mean(Zxx2nor))
# print(np.max(tophase),np.min(tophase),np.mean(tophase))

In [ ]:
from scipy import signal
fs = 360000
for i in range(100):
    freq, times, Zxx = signal.stft(datas[i], fs=fs, nperseg=257, noverlap=128)
    freq = freq[1:]
    Zxx = Zxx[1:]
    # print(freq.shape)
    # print(times.shape)
    # print(Zxx.shape)

    # print("\n")

    # print(np.mean(abs(Zxx)))
    # print(np.max(abs(Zxx)))
    # print(np.min(abs(Zxx)))

    # print("\n")

    fs = 360000
    lownum = 1e-8
    Zxx2 = abs(Zxx) + lownum
    Zxx2log = np.log10(Zxx2)
    # print(np.mean(Zxx2log))
    # print(np.max(Zxx2log))
    # print(np.min(Zxx2log))

    print("\n")

    minva = -4
    Zxx2nor = ( Zxx2log - (minva) )/abs(minva)
    print(np.mean(Zxx2nor))
    print(np.max(Zxx2nor))
    print(np.min(Zxx2nor))


In [3]:
# training start
import Ipynb_importer
# %load train.py
import torch
from torch import autograd
from torch import optim
import json
#from utils import save_samples
import numpy as np
import pprint
import pickle
import datetime
from specgan import *
from utils import *
from logger import *
cuda = True if torch.cuda.is_available() else False

importing Jupyter notebook from specgan.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from config.ipynb
importing Jupyter notebook from logger.ipynb


In [4]:
# =============Logger===============
LOGGER = logging.getLogger('wavegan')
LOGGER.setLevel(logging.DEBUG)

LOGGER.info('Initialized logger.')
init_console_logger(LOGGER)

# =============Parameters===============
args = parse_arguments()
epochs = args['num_epochs']
batch_size = args['batch_size']
latent_dim = args['latent_dim']
ngpus = args['ngpus']
model_size = args['model_size']
model_dir = make_path(os.path.join(args['output_dir'],
                                   datetime.datetime.now().strftime("%Y%m%d%H%M%S")))
args['model_dir'] = model_dir
# save samples for every N epochs.
epochs_per_sample = args['epochs_per_sample']
# gradient penalty regularization factor.
lmbda = args['lmbda']

# Dir
audio_dir = args['audio_dir']
output_dir = args['output_dir']

In [5]:
netG = SpecGANGenerator(model_size=model_size, ngpus=ngpus, latent_dim=latent_dim, upsample=True)
netD = SpecGANDiscriminator(model_size=model_size, ngpus=ngpus)


if cuda:
    netG = torch.nn.DataParallel(netG).cuda()
    netD = torch.nn.DataParallel(netD).cuda()

# if pretrained 
# netD_path = os.path.join(output_dir, "200discriminator.pkl")
# netG_path = os.path.join(output_dir, "200generator.pkl")

# netG.load_state_dict(torch.load(netG_path))
# netD.load_state_dict(torch.load(netD_path))

# "Two time-scale update rule"(TTUR) to update netD 4x faster than netG.
optimizerG = optim.Adam(netG.parameters(), lr=args['learning_rate'], betas=(args['beta1'], args['beta2']))
optimizerD = optim.Adam(netD.parameters(), lr=args['learning_rate'], betas=(args['beta1'], args['beta2']))

In [6]:
# Sample noise used for generated output.
sample_noise = torch.randn(args['sample_size'], latent_dim)
if cuda:
    sample_noise = sample_noise.cuda()
sample_noise_Var = autograd.Variable(sample_noise, requires_grad=False)

# Save config.
LOGGER.info('Saving configurations...')
config_path = os.path.join(model_dir, 'config.json')
with open(config_path, 'w') as f:
    json.dump(args, f)

[INFO] Saving configurations...


In [7]:
print('Loading audio data...')

Loading audio data...


In [8]:
LOGGER.info('Loading audio data...')

# Load cs09 data #
# audio_paths = get_all_audio_filepaths(audio_dir) #list
# train_data, valid_data, test_data, train_size = split_data(audio_paths, args['valid_ratio'],
#                                                            args['test_ratio'], batch_size)

# Load noise data #
num_ = len(real_datas)
num_valid = int(np.ceil(num_ * args['valid_ratio']))
num_train = num_ - num_valid
train_size = num_train
if not (num_valid > 0 and num_train > 0):
    LOGGER.error("Please download DATASET !")
train_data = batch_generator( real_datas[:num_train],batch_size )
valid_data = batch_generator( real_datas[num_train:],batch_size )

TOTAL_TRAIN_SAMPLES = train_size
BATCH_NUM = TOTAL_TRAIN_SAMPLES // batch_size
train_iter = iter(train_data)
valid_iter = iter(valid_data)
#test_iter = iter(test_data)

history = []
D_costs_train = []
D_wasses_train = []
D_costs_valid = []
D_wasses_valid = []
G_costs = []

pkl = 0

[INFO] Loading audio data...


In [ ]:
#epochs = 300
pre_epoch =0
start = time.time()
LOGGER.info('Starting training...EPOCHS={}, BATCH_SIZE={}, BATCH_NUM={}'.format(epochs, batch_size, BATCH_NUM))
for epoch in range(1+pre_epoch, epochs+1):
    LOGGER.info("{} Epoch: {}/{}".format(time_since(start), epoch, epochs))

    D_cost_train_epoch = []
    D_wass_train_epoch = []
    D_cost_valid_epoch = []
    D_wass_valid_epoch = []
    G_cost_epoch = []
    for i in range(1, BATCH_NUM+1):
        for p in netD.parameters():
            p.requires_grad = True

        one = torch.tensor(1, dtype=torch.float)
        neg_one = one * -1
        if cuda:
            one = one.cuda()
            neg_one = neg_one.cuda()

        #######################################################
        # (3) Train Discriminator
        #######################################################
        for iter_dis in range(5): # 5D 1G
            netD.zero_grad()
            noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1)
            if cuda:
                noise = noise.cuda()
            noise_Var = Variable(noise, requires_grad=False)
            real_data_Var = numpy_to_var(next(train_iter)['X'], cuda) 

            # a) compute loss contribution from real training data
            D_real = netD(real_data_Var)
            D_real = D_real.mean()  # avg loss 
            D_real.backward(neg_one)  # loss * -1    
            
            # b) compute loss contribution from generated data, then backprop.
            fake = autograd.Variable(netG(noise_Var).data)
            #print(np.shape(fake))
            D_fake = netD(fake)
            D_fake = D_fake.mean()
            D_fake.backward(one)
            
            # c) compute gradient penalty and backprop
            gradient_penalty = calc_gradient_penalty(netD, real_data_Var.data,
                                                     fake.data, batch_size, lmbda,
                                                     use_cuda=cuda)
            gradient_penalty.backward(one)

            # Compute cost * Wassertein loss..
            D_cost_train = D_fake - D_real + gradient_penalty
            D_wass_train = D_real - D_fake 
            
            # Update gradient of discriminator.
            optimizerD.step()
            

            

            
            
            #############################
            # (2) Compute Valid data
            #############################
            netD.zero_grad()

            valid_data_Var = numpy_to_var(next(valid_iter)['X'], cuda)
            D_real_valid = netD(valid_data_Var)
            D_real_valid = D_real_valid.mean()  # avg loss

            # b) compute loss contribution from generated data, then backprop.
            fake_valid = netG(noise_Var)
            D_fake_valid = netD(fake_valid)
            D_fake_valid = D_fake_valid.mean()

            # c) compute gradient penalty and backprop
            gradient_penalty_valid = calc_gradient_penalty(netD, valid_data_Var.data,
                                                           fake_valid.data, batch_size, lmbda,
                                                           use_cuda=cuda)
            # Compute metrics and record in batch history.
            D_cost_valid = D_fake_valid - D_real_valid + gradient_penalty_valid
            D_wass_valid = D_real_valid - D_fake_valid

            if cuda:
                D_cost_train = D_cost_train.cpu()
                D_wass_train = D_wass_train.cpu()
                D_cost_valid = D_cost_valid.cpu()
                D_wass_valid = D_wass_valid.cpu()

            # Record costs
            D_cost_train_epoch.append(D_cost_train.data.numpy())
            D_wass_train_epoch.append(D_wass_train.data.numpy())
            D_cost_valid_epoch.append(D_cost_valid.data.numpy())
            D_wass_valid_epoch.append(D_wass_valid.data.numpy())
            

        #######################################################
        # (3) Train Generator
        #######################################################
        # Prevent discriminator update.
        for p in netD.parameters():
            p.requires_grad = False

        # Reset generator gradients
        netG.zero_grad()

        # Noise
        noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1)
        if cuda:
            noise = noise.cuda()
        noise_Var = Variable(noise, requires_grad=False) 

        fake = netG(noise_Var)
        G = netD(fake)
        G = G.mean()

        # Update gradients.
        G.backward(neg_one)
        G_cost = -G

        optimizerG.step()

        # Record costs
        if cuda:
            G_cost = G_cost.cpu()
        G_cost_epoch.append(G_cost.data.numpy())

        LOGGER.info("{} Epoch={} Batch: {}/{} D_c:{:.4f} | D_w:{:.4f} | G:{:.4f}".format(time_since(start), epoch,
                                                                                             i, BATCH_NUM,
                                                                                             D_cost_train.data.numpy(),
                                                                                             D_wass_train.data.numpy(),
                                                                                             G_cost.data.numpy()))


    # Save the average cost of batches in every epoch.
    D_cost_train_epoch_avg = sum(D_cost_train_epoch) / float(len(D_cost_train_epoch))
    D_wass_train_epoch_avg = sum(D_wass_train_epoch) / float(len(D_wass_train_epoch))
    D_cost_valid_epoch_avg = sum(D_cost_valid_epoch) / float(len(D_cost_valid_epoch))
    D_wass_valid_epoch_avg = sum(D_wass_valid_epoch) / float(len(D_wass_valid_epoch))
    G_cost_epoch_avg = sum(G_cost_epoch) / float(len(G_cost_epoch))

    D_costs_train.append(D_cost_train_epoch_avg)
    D_wasses_train.append(D_wass_train_epoch_avg)
    D_costs_valid.append(D_cost_valid_epoch_avg)
    D_wasses_valid.append(D_wass_valid_epoch_avg)
    G_costs.append(G_cost_epoch_avg)
    

    LOGGER.info("{} D_cost_train:{:.4f} | D_wass_train:{:.4f} | D_cost_valid:{:.4f} | D_wass_valid:{:.4f} | "
                "G_cost:{:.4f}".format(time_since(start),
                                       D_cost_train_epoch_avg,
                                       D_wass_train_epoch_avg,
                                       D_cost_valid_epoch_avg,
                                       D_wass_valid_epoch_avg,
                                       G_cost_epoch_avg))
    

    #Generate audio samples.
#     if epoch % epochs_per_sample == 0:
#         LOGGER.info("Generating samples...")
#         sample_out = netG(sample_noise_Var)  # sample_noise_Var = (10,100)
#         if cuda:
#             sample_out = sample_out.cpu()
#         sample_out = sample_out.data.numpy() # sample_out = (10,1,128,128)
#         sample_out = np.squeeze(sample_out) 
#         # save spectrogram and timeseries and statistic
#         sample_dir = save_and_plot( times, freq, sample_out, epoch, output_dir )
        
    if epoch % 20 ==0:
        # Save model
        LOGGER.info("Saving models...")
        netD_path = os.path.join(output_dir, str(epoch) + "discriminator.pkl")
        netG_path = os.path.join(output_dir, str(epoch) + "generator.pkl")
        torch.save(netD.state_dict(), netD_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
        torch.save(netG.state_dict(), netG_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
        pkl+=1
        
        
        

LOGGER.info('>>>>>>>Training finished !<<<<<<<')

# Save model
LOGGER.info("Saving models...")
netD_path = os.path.join(output_dir, "discriminator.pkl")
netG_path = os.path.join(output_dir, "generator.pkl")
torch.save(netD.state_dict(), netD_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)
torch.save(netG.state_dict(), netG_path, pickle_protocol=pickle.HIGHEST_PROTOCOL)

# Plot loss curve.
LOGGER.info("Saving loss curve...")
plot_loss(D_costs_train, D_wasses_train,
          D_costs_valid, D_wasses_valid, G_costs, output_dir)

LOGGER.info("All finished!")
        

[INFO] Starting training...EPOCHS=200, BATCH_SIZE=64, BATCH_NUM=220
[INFO] 0m 0s Epoch: 1/200
[INFO] 0m 2s Epoch=1 Batch: 1/220 D_c:-2506.4004 | D_w:2506.5493 | G:1252.5160
[INFO] 0m 5s Epoch=1 Batch: 2/220 D_c:-2518.9312 | D_w:2519.3267 | G:1259.2800
[INFO] 0m 8s Epoch=1 Batch: 3/220 D_c:-2532.0815 | D_w:2532.5776 | G:1265.5997
[INFO] 0m 11s Epoch=1 Batch: 4/220 D_c:-2543.0740 | D_w:2543.3296 | G:1271.8052
[INFO] 0m 14s Epoch=1 Batch: 5/220 D_c:-2558.5403 | D_w:2558.9036 | G:1278.9902
[INFO] 0m 17s Epoch=1 Batch: 6/220 D_c:-2569.4050 | D_w:2569.6396 | G:1285.2880
[INFO] 0m 20s Epoch=1 Batch: 7/220 D_c:-2584.2527 | D_w:2584.5742 | G:1291.8766
[INFO] 0m 23s Epoch=1 Batch: 8/220 D_c:-2597.5647 | D_w:2598.0347 | G:1298.5381
[INFO] 0m 26s Epoch=1 Batch: 9/220 D_c:-2608.0103 | D_w:2608.7666 | G:1304.0209
[INFO] 0m 29s Epoch=1 Batch: 10/220 D_c:-2623.5872 | D_w:2623.8281 | G:1311.7992
[INFO] 0m 32s Epoch=1 Batch: 11/220 D_c:-2638.3049 | D_w:2638.4622 | G:1318.8602
[INFO] 0m 35s Epoch=1 Batch

[INFO] 5m 1s Epoch=1 Batch: 102/220 D_c:-3921.3442 | D_w:3921.4875 | G:1963.6973
[INFO] 5m 4s Epoch=1 Batch: 103/220 D_c:-3936.4109 | D_w:3936.5823 | G:1971.0387
[INFO] 5m 7s Epoch=1 Batch: 104/220 D_c:-3949.2078 | D_w:3950.0293 | G:1976.4196
[INFO] 5m 10s Epoch=1 Batch: 105/220 D_c:-3965.0132 | D_w:3965.2400 | G:1985.5021
[INFO] 5m 13s Epoch=1 Batch: 106/220 D_c:-3980.6873 | D_w:3980.8125 | G:1993.4407
[INFO] 5m 16s Epoch=1 Batch: 107/220 D_c:-3994.2869 | D_w:3995.2080 | G:1999.4357
[INFO] 5m 19s Epoch=1 Batch: 108/220 D_c:-4010.2566 | D_w:4010.6245 | G:2008.2893
[INFO] 5m 22s Epoch=1 Batch: 109/220 D_c:-4024.2400 | D_w:4024.7109 | G:2015.3965
[INFO] 5m 25s Epoch=1 Batch: 110/220 D_c:-4040.0762 | D_w:4040.8022 | G:2023.3635
[INFO] 5m 28s Epoch=1 Batch: 111/220 D_c:-4053.8845 | D_w:4054.4414 | G:2028.2991
[INFO] 5m 31s Epoch=1 Batch: 112/220 D_c:-4067.2810 | D_w:4067.8677 | G:2037.9238
[INFO] 5m 34s Epoch=1 Batch: 113/220 D_c:-4087.1147 | D_w:4087.4097 | G:2047.4490
[INFO] 5m 37s Epoch

# generate cyclo noise

In [ ]:
print(epoch)

In [ ]:
LOGGER.info("Generating samples...")
fakee = []
for i in range(245):
    noise = torch.Tensor(batch_size, latent_dim).uniform_(-1, 1)
    if cuda:
        noise = noise.cuda()
    noise_Var = Variable(noise, requires_grad=False)
    sample_out = netG(noise_Var)  
    if cuda:
        sample_out = sample_out.cpu()
    sample_out = sample_out.data.numpy() # sample_out = (10,1,128,128)
    sample_out = np.squeeze(sample_out)
    fakee.append(sample_out)
fakee = np.reshape( np.array(fakee),(-1,128,128) )
print(fakee.shape)

In [ ]:
print(fakee.shape)

In [ ]:
########### MIX GAN ############

In [ ]:
#load cyclcos data

import csv
import numpy as np
Wavedatas = []
dataname = 'w_nearest_ep200(cyclo2_fresh)'
with open( dataname +'.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        Wavedatas.append( list( map(float,row) ) )
    csvfile.close()
Wavedatas = np.array(Wavedatas)
print(Wavedatas.shape)

In [ ]:
f1, t1, Zxx1 = signal.stft(Wavedatas[i], fs=400000, nperseg=257, noverlap=128)
print(Zxx1.shape)

In [ ]:
freqs = 400000
g_iters = 15

def _stft_(y):
    f, t, Zxx = signal.stft(y, fs=freqs, nperseg=257, noverlap=128)
    return Zxx

def _griffin_lim(S,ang):
    m = np.mean(S,axis=0)
    S = np.vstack((m,S))
    angles = ang
    S_complex = np.abs(S).astype(np.complex)
    
#     for i in range(g_iters):
#         if i > 0:
#             angles = np.exp(1j * np.angle(_stft_(y[1])))
#         y = _istft_( S_complex * np.exp(1j*angles) )
    y = _istft_( S_complex * np.exp(1j*angles) )
    return y[1]

def _istft_(S):
    return signal.istft(S, fs=freqs, nperseg=257, noverlap=128)

def spec_to_time(spectrogram,angle):
    minva = -4
    lownum = 1e-8
    Zxx2denor = spectrogram*abs(minva) + minva
    Zxx2dbtoamp = np.power(10,Zxx2denor)- lownum
    estimate = _griffin_lim(Zxx2dbtoamp, ang)
    return estimate

In [ ]:
f1, t1, Zxx1 = signal.stft(Wavedatas[1], fs=400000, nperseg=257, noverlap=128)
ang = np.angle(Zxx1)
a = spec_to_time(fakee[1], ang)

In [ ]:
fake_datas = []
for i in range(len(fakee)):
    print(i)
    f1, t1, Zxx1 = signal.stft(Wavedatas[i], fs=400000, nperseg=257, noverlap=128)
    ang = np.angle(Zxx1)
    fake_datas.append( spec_to_time(fakee[i], ang) )
fake_datas = np.array(fake_datas)
print(fake_datas.shape)

In [ ]:
save = 'metrice'
if not os.path.isdir(save):
    os.makedirs(save)

In [ ]:
a = save + "/" + "mixgan_nogriffin_ep200" + "(cyclo2_fresh)" + ".csv"
with open(a,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(fake_datas)): 
        csvWriter.writerow( fake_datas[i] ) 

In [ ]:
############ end #################

In [ ]:
# spectrogram
cm=plt.cm.get_cmap('rainbow') 
plt.figure(figsize=(30,10))
plt.pcolormesh(fakee[0],cmap=cm)

In [ ]:
# time_series
plt.plot(spectrogram_to_time(fakee[0])[1])

In [ ]:

fake_datas = []
for i in range(len(fakee)):
    print(i)
    fake_datas.append( spectrogram_to_time(fakee[i])[1] )
fake_datas = np.array(fake_datas)

In [ ]:
####################### validate ######################

In [ ]:
fake_datas = []
l = len(fakee)
for i in range(len(fakee)):
    print(i)
    fake_datas.append( spectrogram_to_time(fakee[i])[1] )
fake_datas = np.array(fake_datas)

In [ ]:
import numpy as np
from scipy import signal
#real_datas = np.array(real_datas[:15000])
fake_datas = np.array(fake_datas[:15000])
#print(real_datas.shape)
print(fake_datas.shape)

In [ ]:
# create a new folder to store metrice

save = 'metrice'
if not os.path.isdir(save):
    os.makedirs(save)

In [ ]:
a = save + "/" + "spec_ep200" + "(cyclo2_fresh)" + ".csv"
with open(a,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(fake_datas)): 
        csvWriter.writerow( fake_datas[i] ) 
        


In [ ]:
# paremeter
fs = 400000 # 400k
data_length = len(real_datas[0])

In [ ]:
# draw time series

import matplotlib.pyplot as plt
#fs = 360000 # 360k
t = data_length / fs # sampling freq * sampling time = sample num

plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.plot( np.arange(0,t,1/fs), real_datas[500], label = 'Real',color = (1,0,0))
plt.legend(loc = 'upper right',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Noise amplitude(v)",fontsize=16)

plt.subplot(2,1,2)
plt.plot( np.arange(0,t,1/fs), fake_datas[2], label = 'Generate')
plt.legend(loc = 'upper right',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Noise amplitude(v)",fontsize=16)


plt.savefig(save + '/' + "time_series.png")

In [ ]:
x = np.random.randint(1,15000, size=(10))
tot = 1
# draw random 10 time series

import matplotlib.pyplot as plt
#fs = 360000 # 360k
t = data_length / fs # sampling freq * sampling time = sample num

plt.figure(figsize=(15,50))
for i in x:
    plt.subplot(20,1,tot)
    plt.plot( np.arange(0,t,1/fs), real_datas[i], label = 'Real',color = (1,0,0))
    plt.legend(loc = 'upper right',fontsize=16)
    plt.xlabel("Time(s)",fontsize=16)
    plt.ylabel("Noise amplitude(v)",fontsize=10)
    
    tot += 1
    
    plt.subplot(20,1,tot)
    plt.plot( np.arange(0,t,1/fs), fake_datas[i], label = 'Generate')
    plt.legend(loc = 'upper right',fontsize=16)
    plt.xlabel("Time(s)",fontsize=16)
    plt.ylabel("Noise amplitude(v)",fontsize=10)
    
    tot += 1
    
plt.savefig(save + '/' + "time_series(10random).png")

In [ ]:
# draw stft
from scipy import signal
# fs = 360000
f1, t1, Zxx1 = signal.stft(real_datas[500], fs=fs, nperseg=257, noverlap=128)
f2, t2, Zxx2 = signal.stft(fake_datas[1], fs=fs, nperseg=257, noverlap=128)

import matplotlib.pyplot as plt
cm=plt.cm.get_cmap('plasma')
plt.figure(figsize=(20,10))
#plt.title(" X:Time[s], Y:Frequency[KHz]")
plt.subplot(2,1,1)
plt.pcolormesh(t1, f1/1000, abs(Zxx1),cmap=cm)
plt.colorbar()
#plt.legend(loc = 'best',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Frequency(kHz)",fontsize=16)

plt.subplot(2,1,2)
plt.pcolormesh(t2, f2/1000, abs(Zxx2),cmap=cm)
plt.colorbar()
#plt.legend(loc = 'best',fontsize=16)
plt.xlabel("Time(s)",fontsize=16)
plt.ylabel("Frequency(kHz)",fontsize=16)
plt.savefig(save + '/' + "stft.png")

In [ ]:
# check frequency value
print(np.where(Zxx1>0.20))
print(f1[43]/1000)

In [ ]:
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
le = len(real_datas[2])
xf = np.linspace(0,fs,data_length) # 頻率
xf2 = xf[:(le//2)]
r_yf2 = abs(fft(fake_datas[3]))[:(le//2)]
plt.plot(xf2,r_yf2)
print(r_yf2.shape)
print(xf2)

In [ ]:
# Average FFT

from scipy.fftpack import fft,ifft
r_average_fft = np.zeros(len(real_datas[0]))
f_average_fft = np.zeros(len(fake_datas[0]))
le = len(real_datas)
for i in range(len(real_datas)):
    r_average_fft += abs( fft(real_datas[i]) ) / le
    f_average_fft += abs( fft(fake_datas[i]) ) / le
    
ln = len(real_datas[0])
import matplotlib.pyplot as plt
r_yf2 = r_average_fft[ : (ln//2) ]
f_yf2 = f_average_fft[ : (ln//2) ]
xf = np.linspace(0,fs,data_length) # 頻率
xf2 = xf[:(ln//2)] / 1000

#print( r_average_fft.shape)

plt.figure(figsize=(20,10))
plt.plot(xf2,r_yf2,label = 'Real',color=(1,0,0))
plt.plot(xf2,f_yf2,label = 'Generate',alpha=0.5)
plt.legend(loc='best',fontsize=16)
plt.title('FFT of Mixed wave',fontsize=20,color='#F08080')
plt.xlabel("Frequency(kHz)",fontsize=16)
plt.ylabel("Average FFT",fontsize=16)

plt.savefig(save + '/' + "fft.png")

In [ ]:
print( np.where(f_yf2>200) )
print( xf[2707]/1000 )

In [ ]:
# statisitc - easy version

from scipy import stats
from scipy.signal import find_peaks

def feature_change(chandata):
    total_data = []
    print(len(chandata))
    for i in range(len(chandata)):
        print(i)
        # statistic data
        nobs,(damin,damax),damean,davar,daskew,dakurto = stats.describe( chandata[i] )
        total_data.append( [damax, damean, davar**(1/2), daskew, dakurto] )
    return np.array( total_data )

rdata_statis = feature_change(real_datas)
fdata_statis = feature_change(fake_datas)

fdata_statis = fdata_statis[:15675]
print(rdata_statis.shape)
print(fdata_statis.shape)

In [ ]:
# real datas easy version

total_mean = np.mean(rdata_statis, axis = 0)
total_std = np.std(rdata_statis, axis = 0)
total_median = np.median(rdata_statis, axis = 0)

print("max[v], mean[mv], std[v], skewness, kurtosis")
print(total_mean)
print(total_std)
print(total_median)

with open('real_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[mv]', 'std[v]', 'skewness', 'kurtosis'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()
    

#fake datas

total_mean = np.mean(fdata_statis, axis = 0)
total_std = np.std(fdata_statis, axis = 0)
total_median = np.median(fdata_statis, axis = 0)

print("max[v], mean[mv],std[v], skewness, kurtosis")
print(total_mean)
print(total_std)
print(total_median)
    
with open('fake_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[mv]', 'std[v]', 'skewness', 'kurtosis'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()

In [ ]:
# statisitc

from scipy import stats
from scipy.signal import find_peaks

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size//2:]

def feature_change(chandata):
    total_data = []
    print(len(chandata))
    for i in range(len(chandata)):
        print(i)
        # statistic data
        nobs,(damin,damax),damean,davar,daskew,dakurto = stats.describe( chandata[i] )
        # energy
        E_fft = np.sum(chandata[i] ** 2) / len(chandata[i])
        # auto statistic data
        nobs,(aumin,aumax),aumean,auvar,auskew,aukurto = stats.describe( autocorr(chandata[i]) ) 
        # peak > 0.05
        peaks, _ = find_peaks(chandata[i], height=0.05)
        
        total_data.append( [damax, damean, E_fft, davar**(1/2), daskew, dakurto, int(len(peaks)), auskew, aukurto] )
    return np.array( total_data )

rdata_statis = feature_change(real_datas)
fdata_statis = feature_change(fake_datas)

print(rdata_statis.shape)
print(fdata_statis.shape)

In [ ]:
print(rdata_statis.shape)
print(fdata_statis.shape)

In [ ]:
# store random 100 datas to generate CPS and coherence

a = save + "/" + "spec_ep200_statistic" + "(cyclo2_fresh)" + ".csv"
with open(a,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(fdata_statis)):
        csvWriter.writerow( fdata_statis[i] ) 
        

        
b = save + "/" + "real_statistic" + "(cyclo2_fresh)" + ".csv"
with open(b,'w',newline='') as csvFile:
    csvWriter = csv.writer(csvFile)
    for i in range(len(rdata_statis)):
        csvWriter.writerow( rdata_statis[i] )

In [ ]:
# real datas

total_mean = np.mean(rdata_statis, axis = 0)
total_std = np.std(rdata_statis, axis = 0)
total_median = np.median(rdata_statis, axis = 0)

print("max[v], mean[v], energy[J], std[v], skewness, kurtosis, peak>0.05, skewauto, kuroauto")
print(total_mean)
print(total_std)
print(total_median)

with open(save + '/' + 'real_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[v]', 'energy[J]', 'std[v]', 'skewness', 'kurtosis', 'peak>0.05', 'skewauto', 'kuroauto'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()
    

#fake datas

total_mean = np.mean(fdata_statis, axis = 0)
total_std = np.std(fdata_statis, axis = 0)
total_median = np.median(fdata_statis, axis = 0)

print("max[v], mean[v], energy[J], std[v], skewness, kurtosis, peak>0.05, skewauto, kuroauto")
print(total_mean)
print(total_std)
print(total_median)
    
with open(save + '/' + 'fake_statistic.csv','w',newline="") as csvfile:
    outwriter = csv.writer(csvfile)
    outwriter.writerow(['feature']+['max[v]', 'mean[v]', 'energy[J]', 'std[v]', 'skewness', 'kurtosis', 'peak>0.05', 'skewauto', 'kuroauto'])
    outwriter.writerow(['mean']+list(total_mean))
    outwriter.writerow(['std']+list(total_std))
    outwriter.writerow(['median']+list(total_median))
    csvfile.close()

In [ ]:
# normalize data

#r_norm = np.linalg.norm(rdata_statis,axis=0)
r_max = np.max(rdata_statis,axis=0)
r_min = np.min(rdata_statis, axis = 0)
r_mean = np.mean(rdata_statis, axis = 0)
rdata_statis_norm = np.zeros(np.shape(rdata_statis))
for i in range(rdata_statis_norm.shape[1]):
    rdata_statis_norm[:,i] = ( (rdata_statis[:,i]-r_mean[i]) )/(r_max[i]-r_min[i])


#f_norm = np.linalg.norm(fdata_statis,axis=0)
f_max = np.max(fdata_statis,axis=0)
f_min = np.min(fdata_statis, axis = 0)
f_mean = np.mean(fdata_statis, axis = 0)
fdata_statis_norm = np.zeros(np.shape(fdata_statis))
for i in range(fdata_statis_norm.shape[1]):
    fdata_statis_norm[:,i] = ( (fdata_statis[:,i]-f_mean[i]) )/(f_max[i]-f_min[i])

print(rdata_statis_norm.shape)
print(fdata_statis_norm.shape)


# mix

dataplot = []
for i in range(9):
    dataplot.append(rdata_statis_norm[:,i])
    dataplot.append(fdata_statis_norm[:,i])
dataplot = np.transpose(np.array(dataplot))
print(dataplot.shape)

labels=[]
for i in range(1,10):
    labels.append(str(i)+"")
    labels.append('*')

fig,axes = plt.subplots()
axes.boxplot(x=dataplot,sym='rd',patch_artist=True,labels=labels)
axes.set_xlabel('Feature')
plt.text(14,0.9,'Num : Real')
plt.text(15,0.8,'* : Generate')
plt.savefig(save + '/' + "boxplot.png")